In [55]:
#!pip install schedule
import pandas as pd
import pickle
import schedule
import time
import threading
from datetime import datetime
from vol_data import *
from surface import *

### Inputs

In [69]:
# Define your parameters
spe_time = "22:48"

start_date = '2024-01-01'
end_date = '2024-05-28'

udl_list = ['JP_NKY', 'DE_DAX', 'GB_FTSE100', 'CH_SMI', 'IT_FTMIB', 'ES_IBEX', 'US_SPX', 'EU_STOXX50E', 'EU_SX7E', 'EU_SX7P',
            'EU_SXDP', 'US_KO', 'US_MCD', 'US_KOMO', 'EU_SXPP', 'EU_SOXP', 'HK_HSI']

matu_list = [1, 3, 6, 9, 12, 18, 24, 36]
moneyness_list = [120, 110, 105, 102.5, 100, 97.5, 95, 90, 80]
delta_list = [5, 10, 15, 20, 25, 35, 50, 65, 75, 90, 95]

path = "vol_surf.pickle"  # Path to save the pickle file

### Functions

In [70]:
def get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date):
    """
    Generate random volatility moneyness data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, mon): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for mon in moneyness_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'moneyness'])
    return df

def get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date):
    """
    Generate random volatility delta data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, delta): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for delta in delta_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'delta'])
    return df
    
def generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date):
    """
    Generate random volatility data for moneyness and delta in a single step and create a structured DataFrame.

    Args:
        udl_list (list): List of underlying assets.
        matu_list (list): List of maturities.
        moneyness_list (list): List of moneyness levels.
        delta_list (list): List of delta levels.
        start_date (str): Start date for the data.
        end_date (str): End date for the data.

    Returns:
        DataFrame: DataFrame with combined moneyness and delta data, indexed by date with MultiIndex columns.
    """
    try:
        date_range = pd.date_range(start=start_date, end=end_date, freq='B')

        # Generate moneyness and delta data
        df_vol_moneyness = get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date)
        df_vol_delta = get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date)

        data = {}

        # Combine moneyness data
        for udl in udl_list:
            for matu in matu_list:
                for mon in moneyness_list:
                    key = (udl, matu, mon)
                    data[(udl, 'IV', matu, mon)] = df_vol_moneyness[key]

        # Combine delta data
        for udl in udl_list:
            for matu in matu_list:
                for delta in delta_list:
                    key = (udl, matu, delta)
                    data[(udl, 'IVFD', matu, delta)] = df_vol_delta[key]

        df = pd.DataFrame(data, index=date_range)
        df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'param', 'matu', 'value'])
        df.index.name = 'Date'
        df.ffill(inplace=True)

        return df
    except Exception as e:
        print(f"An error occurred during table generation: {e}")
        return pd.DataFrame()

def df_to_nested_dict(df):
    nested_dict = {}
    for date, row in df.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        nested_dict[date_str] = {}
        for col, val in row.items():
            udl, param, matu, value = col
            nested_dict[date_str].setdefault(udl, {}).setdefault(param, {}).setdefault(matu, {})[value] = val
    return nested_dict

def generate_and_save_vol_surf():
    try:
        # Generate the table
        df = generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date)

        # Convert DataFrame to nested dictionary
        nested_dict = df_to_nested_dict(df)

        # Save the nested dictionary to a pickle file
        with open(path, 'wb') as handle:
            pickle.dump(nested_dict, handle)

        print(f"{datetime.now()} - Volatility surface data generated and saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)  # Wait a bit before checking again

### Exec

In [71]:
# Schedule the job to run at midnight every day
schedule.every().day.at(spe_time).do(generate_and_save_vol_surf)

# Start the scheduler in a new thread
scheduler_thread = threading.Thread(target=run_schedule, daemon=True)
scheduler_thread.start()

### Print

In [73]:
print("Scheduler started. The task will run every day at: " + spe_time)

Scheduler started. The task will run every day at: 22:48
2024-05-27 22:48:01.210940 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:01.662461 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:02.050020 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:02.432224 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:03.090510 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:03.546454 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:04.080194 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:04.096843 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:04.787595 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:05.431505 - Volatility surface data generated and saved successfully.
2024-05-27 22:48:06.539297 - Volatility surface data generated and saved 

Exception in thread Exception in thread Thread-5 (run_schedule):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
Thread-9 (run_schedule):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y0/5dv_k1x105l2r3xvbg3mwj3h0000gn/T/ipykernel_28713/984865960.py", line 20, in run_schedule
    self.run()
  File "/opt/anaconda3/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/y0/5dv_k1x105l2r3xvbg3mwj3h0000gn/T/ipykernel_28713/984865960.py", line 20, in run_schedule
NameError: name 'time' is not defined
NameError: name 'time' is not defined


### To go further

In [11]:
# Generate table
df = generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date)

# Convert DataFrame to nested dictionary and back to DataFrame
nested_dict = df_to_nested_dict(df)
reconstructed_df = nested_dict_to_df(nested_dict)

# Check if the reconstructed DataFrame is equal to the original
comparison = df.equals(reconstructed_df)
print(f"DataFrames are equal: {comparison}")

if not comparison:
    print("Differences found between original and reconstructed DataFrames:")
    print("Original DataFrame head:")
    print(df.head())
    print("Reconstructed DataFrame head:")
    print(reconstructed_df.head())

    print("Original DataFrame index and columns:")
    print(df.index)
    print(df.columns)

    print("Reconstructed DataFrame index and columns:")
    print(reconstructed_df.index)
    print(reconstructed_df.columns)

# SANITY CHECK - Check if the reconstructed DataFrame is equal to the original
comparison = df.equals(reconstructed_df)
print(f"DataFrames are equal: {comparison}")

if not comparison:
    print("Differences found between original and reconstructed DataFrames:")
    print("Original DataFrame head:")
    print(df.head())
    print("Reconstructed DataFrame head:")
    print(reconstructed_df.head())
    
    print("Original DataFrame index and columns:")
    print(df.index)
    print(df.columns)
    
    print("Reconstructed DataFrame index and columns:")
    print(reconstructed_df.index)
    print(reconstructed_df.columns)

# Set initial visibility
toggle_date_widgets(surface_type_widget.value)

# Display the widgets and output
display(main_box)

# Function to toggle date widgets visibility
def toggle_date_widgets(surface_type):
    if surface_type in ['Percentile', 'Z-score']:
        start_date_widget.layout.display = 'block'
        end_date_widget.layout.display = 'block'
        date_widget.layout.display = 'none'
    else:
        start_date_widget.layout.display = 'none'
        end_date_widget.layout.display = 'none'
        date_widget.layout.display = 'block'


DataFrames are equal: True
DataFrames are equal: True
